# 1.0 Fix data

AIMS 

En este archivo se realiza la limpieza de los archivos y el ETL

In [10]:
import pandas as pd
from datetime import datetime
from mypackage import dir, dataquality

In [11]:
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
modality =  modality = 'p'
project = 'australia'
data = dir.make_dir_line(modality, project) 
raw = data('raw')

In [13]:
def discrete_values(dataframe, threshold=10):
    # Using Value Counts
    discrete_vars_counts = [col for col in dataframe.columns if dataframe[col].nunique() < threshold]  # Assuming a threshold here

    # Print or use the identified discrete variables
    print("Discrete variables by value counts:", discrete_vars_counts)

def non_sex(dataframe, variable):
    # Label sex or gender
    ''' 
        This function takes a column and returns Label sex or gender.
        0 = Female
        1 = Male
    '''
    dataframe[variable] = dataframe[variable].replace(['F', 'f', 'Femenino', 'femenino'], 0)
    dataframe[variable] = dataframe[variable].replace(['M', 'm', 'Masculino', 'masculino'], 1)

    return dataframe

def diff_date(dataframe, column, date):

    ''' 
        This function takes a column and returns the difference with the current date or the entered date.
    '''

    from datetime import datetime

    if date == 'now':
        current_date = datetime.today()
    else:
        current_date = pd.to_datetime(date)
 
    dataframe['age_days'] = (current_date - dataframe[column]).dt.days

    return dataframe

## 1.1 Fix data customers

In [14]:
df_clientes = pd.read_csv(raw / 'df_clientes.csv', sep = ',', decimal = '.', header = 0)

df_clientes['fecha_nacimiento'] = pd.to_datetime(df_clientes['fecha_nacimiento'], dayfirst='false', errors = 'coerce')

dataquality.data_quality(df_clientes)

discrete_values(df_clientes, 10)

*****
The dataframe does not contain missing values.
*****
The dataframe does not contain duplicate values.
*****
The dataframe contains values that are not numeric.
Variable not numerit: nombre
Variable not numerit: sexo
Variable not numerit: fecha_nacimiento
*****
The dimension of dataframe is (100, 4)
execution data quality ends.
Discrete variables by value counts: ['sexo']


In [15]:
df_clientes = non_sex(df_clientes, 'sexo')
df_clientes = diff_date(df_clientes, "fecha_nacimiento", 'now')
df_clientes.head()

,id_cliente,nombre,sexo,fecha_nacimiento,age_days
0,1001,Fredo Bonanno-Beccheria,1,1930-04-27,34247
1,1002,Ermenegildo Sansoni,1,1940-09-02,30466
2,1003,Antonia Bajardi,0,1916-05-01,39356
3,1004,Evangelista Biagi,0,1985-01-03,14272
4,1005,Iolanda Ceschi,0,2016-06-07,2794


## 1.2 Fix data products

In [16]:
df_productos = pd.read_csv(raw / 'df_productos.csv', sep = ',', decimal = '.', header = 0)
df_productos['production2'] = (df_productos['precio'] * df_productos['production'])/100
dataquality.data_quality(df_productos)
discrete_values(df_productos, 10)

*****
The dataframe does not contain missing values.
*****
The dataframe does not contain duplicate values.
*****
The dataframe contains values that are not numeric.
Variable not numerit: id_producto
Variable not numerit: articulo
Variable not numerit: category
Variable not numerit: color
Variable not numerit: L
Variable not numerit: tallas
*****
The dimension of dataframe is (395, 10)
execution data quality ends.
Discrete variables by value counts: ['tallas']


In [17]:
df_productos.head()

,id_producto,descripcion,articulo,category,precio,production,color,L,tallas,production2
0,759ne,759,Short con tiritas,Short,40.0,54,Negro,ne,1 al 4,21.60
1,759bl,759,Short con tiritas,Short,40.0,54,Blanco,bl,1 al 4,21.60
2,759az,759,Short con tiritas,Short,40.0,54,Azul,az,1 al 4,21.60
3,759ro,759,Short con tiritas,Short,40.0,54,Rojo,ro,1 al 4,21.60
4,761ne,761,Short lienzo con bolsillo fruncido,Short,43.2,45,Negro,ne,1 al 4,19.44


## 1.3 Fix data transactions

In [22]:
df_ventas = pd.read_csv(raw / 'df_ventas.csv', sep = ',', decimal = '.', header = 0)
df_ventas['fecha'] = pd.to_datetime(df_ventas['fecha'], dayfirst='false', errors = 'coerce')
df_ventas['id_factura'] = df_ventas['id_factura'].astype(str)
df_ventas['id_cliente'] = df_ventas['id_cliente'].astype(str)
df_ventas['id_store'] = df_ventas['id_store'].astype(str)
df_ventas['discount'] = df_ventas['discount'].astype(str)
df_ventas['no_cpk'] = df_ventas['no_cpk'].astype(str)
dataquality.data_quality(df_ventas)
discrete_values(df_ventas, 10)

*****
The dataframe does not contain missing values.
*****
The dataframe does not contain duplicate values.
*****
The dataframe contains values that are not numeric.
Variable not numerit: id_factura
Variable not numerit: fecha
Variable not numerit: id_cliente
Variable not numerit: id_producto
Variable not numerit: id_store
Variable not numerit: discount
Variable not numerit: no_cpk
*****
The dimension of dataframe is (5331, 10)
execution data quality ends.
Discrete variables by value counts: ['quantity', 'id_store', 'discount', 'per_discount2', 'no_cpk']


In [19]:
df_ventas.head()

,id_factura,fecha,id_cliente,id_producto,quantity,id_store,discount,per_discount,per_discount2,no_cpk
0,1001,2021-01-31,2088,9169mo,2,0,0,0.0,0.0,0
1,1002,2021-02-01,2076,9156ro,2,1,0,0.0,0.0,0
2,1003,2021-02-02,2051,9168az,2,1,1,8.0,5.0,0
3,1004,2021-02-03,2053,9171mo,2,2,1,8.0,10.0,0
4,1005,2021-02-05,2087,9142am,1,1,0,0.0,0.0,0


In [23]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5331 entries, 0 to 5330
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_factura     5331 non-null   object        
 1   fecha          5331 non-null   datetime64[ns]
 2   id_cliente     5331 non-null   object        
 3   id_producto    5331 non-null   object        
 4   quantity       5331 non-null   int64         
 5   id_store       5331 non-null   object        
 6   discount       5331 non-null   object        
 7   per_discount   5331 non-null   float64       
 8   per_discount2  5331 non-null   float64       
 9   no_cpk         5331 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 416.6+ KB
